# YOLOv8による物体検出 - Google Colab版

このノートブックでは、YOLOv8（You Only Look Once v8）を使用して物体検出を行います。YOLOv8は最新の物体検出モデルで、高精度かつ高速な推論が可能です。

## 特徴
- リアルタイム物体検出
- 高精度な位置特定
- 80種類のオブジェクトクラスに対応
- 画像・動画両方に対応

## 必要な環境
- Google Colab（推奨）
- Python 3.8以上

## 1. 必要なライブラリのインストール

まず、YOLOv8を使用するために必要なライブラリをインストールします。

In [ ]:
# YOLOv8のメインライブラリであるUltralyticsをインストール
!pip install ultralytics

# その他の必要なライブラリをインストール（Google Colabには多くが既にインストール済み）
!pip install opencv-python-headless pillow matplotlib

# インストール完了の確認
print("✅ 全てのライブラリのインストールが完了しました！")

## 2. ライブラリのインポート

物体検出に必要なライブラリをインポートします。

In [ ]:
# 必要なライブラリをインポート
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from google.colab import files
import io

# 日本語フォントの設定（可視化用）
plt.rcParams['font.family'] = 'DejaVu Sans'

print("✅ 全てのライブラリのインポートが完了しました！")

## 3. YOLOv8モデルの読み込み

事前訓練済みのYOLOv8モデルを読み込みます。モデルサイズによって精度と速度が異なります：

- **yolov8n**: 最小サイズ（最高速度、最低精度）
- **yolov8s**: 小サイズ
- **yolov8m**: 中サイズ
- **yolov8l**: 大サイズ
- **yolov8x**: 最大サイズ（最低速度、最高精度）

In [ ]:
# YOLOv8モデルを読み込み（初回実行時は自動でダウンロードされます）
model = YOLO('yolov8n.pt')  # nanoモデルを使用（高速で軽量）

# モデル情報を表示
print("✅ YOLOv8モデルの読み込みが完了しました！")
print(f"モデル名: {model.model_name}")
print(f"クラス数: {len(model.names)}")
print(f"検出可能なクラス: {list(model.names.values())[:10]}...など") # 最初の10個を表示

## 4. テスト画像のアップロードと準備

Google Colabに画像をアップロードして物体検出を実行します。

In [ ]:
# 画像ファイルをアップロード
print("📁 画像ファイルを選択してアップロードしてください...")
uploaded = files.upload()

# アップロードされた画像のパスを取得
uploaded_image_paths = list(uploaded.keys())
print(f"✅ {len(uploaded_image_paths)}個の画像がアップロードされました")

# アップロードされた画像を表示
if uploaded_image_paths:
    fig, axes = plt.subplots(1, min(3, len(uploaded_image_paths)), figsize=(15, 5))
    if len(uploaded_image_paths) == 1:
        axes = [axes]
    
    for i, image_path in enumerate(uploaded_image_paths[:3]):
        img = Image.open(image_path)
        if len(uploaded_image_paths) > 1:
            axes[i].imshow(img)
            axes[i].set_title(f'アップロード画像 {i+1}: {image_path}')
            axes[i].axis('off')
        else:
            axes[0].imshow(img)
            axes[0].set_title(f'アップロード画像: {image_path}')
            axes[0].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ 画像がアップロードされませんでした")

## 5. 画像での物体検出実行

アップロードした画像に対してYOLOv8による物体検出を実行します。

In [ ]:
# 物体検出を実行
if uploaded_image_paths:
    results_list = []
    
    for image_path in uploaded_image_paths:
        print(f"🔍 {image_path} を解析中...")
        
        # YOLOv8で物体検出を実行
        results = model(image_path, conf=0.5)  # 信頼度50%以上の検出結果のみ表示
        results_list.append(results[0])
        
        # 検出結果の詳細を表示
        print(f"✅ 検出完了: {len(results[0].boxes)}個のオブジェクトが検出されました")
        
        # 検出されたオブジェクトの詳細
        if len(results[0].boxes) > 0:
            for i, box in enumerate(results[0].boxes):
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                class_name = model.names[class_id]
                print(f"  - オブジェクト{i+1}: {class_name} (信頼度: {confidence:.2f})")
        else:
            print("  - オブジェクトが検出されませんでした")
        print()
    
    print("🎉 全ての画像の物体検出が完了しました！")
else:
    print("❌ 検出する画像がありません。画像をアップロードしてください。")

## 6. 検出結果の可視化

検出されたオブジェクトにバウンディングボックス、クラスラベル、信頼度スコアを重ねて表示します。

In [ ]:
# 検出結果を可視化
if uploaded_image_paths and results_list:
    
    # 画像数に応じてsubplotを調整
    num_images = len(uploaded_image_paths)
    cols = min(2, num_images)
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 8 * rows))
    if num_images == 1:
        axes = [axes]
    elif rows == 1:
        axes = [axes] if cols == 1 else axes
    else:
        axes = axes.flatten()
    
    for i, (image_path, results) in enumerate(zip(uploaded_image_paths, results_list)):
        # 画像を読み込み
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # 検出結果を画像に描画
        annotated_img = results.plot()
        annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
        
        # 表示
        if num_images > 1:
            axes[i].imshow(annotated_img_rgb)
            axes[i].set_title(f'検出結果: {image_path}')
            axes[i].axis('off')
        else:
            axes[0].imshow(annotated_img_rgb)
            axes[0].set_title(f'検出結果: {image_path}')
            axes[0].axis('off')
    
    # 余った subplot があれば非表示にする
    if num_images < len(axes):
        for j in range(num_images, len(axes)):
            axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("🎨 検出結果の可視化が完了しました！")
    print("📊 バウンディングボックス内の情報:")
    print("   - 色付きの四角: 検出されたオブジェクトの境界")
    print("   - テキスト: クラス名と信頼度スコア")
else:
    print("❌ 表示する結果がありません")